In [207]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import folium
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats
import numpy as np
from math import sqrt

In [212]:
def CalcRSquared(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    
    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)
    
    return RMSE

III.1

In [82]:
cdatasub999 = pd.read_csv("london_flows.csv")

In [83]:
#plus one before log distance because 0 exists in the dataset
x_variables = ["population", "jobs", "distance"]
log_x_vars = []
for x in x_variables:
    cdatasub999[f"log_{x}"] = np.log(cdatasub999[x]+0.01)
    log_x_vars.append(f"log_{x}")

In [90]:
formula = 'flows ~ log_population + log_jobs + log_distance'
uncosim2 = smf.glm(formula = formula, 
                  data=cdatasub999, 
                  family=sm.families.Poisson()).fit()
print(uncosim2.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61474
Model:                            GLM   Df Residuals:                    61470
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.4298e+06
Date:                Mon, 09 May 2022   Deviance:                   2.6875e+06
Time:                        20:23:56   Pearson chi2:                 6.30e+06
No. Iterations:                    14                                         
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -8.9966      0.011   -838.

In [91]:
cdatasub999["uncosim"] = np.round(uncosim2.mu)
#here's the matrix
cdatasubmat_uncon2 = cdatasub999.pivot_table(values ="uncosim", index="station_origin", columns = "station_destination",
                                    aggfunc=np.sum, margins=True)
cdatasubmat_uncon2

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,215.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,347.0
Acton Town,NaN,NaN,NaN,28.0,28.0,NaN,5.0,2.0,NaN,29.0,...,23.0,3.0,9.0,9.0,NaN,4.0,NaN,4.0,NaN,4547.0
Aldgate,NaN,NaN,8.0,NaN,33.0,NaN,NaN,1.0,NaN,29.0,...,19.0,NaN,8.0,6.0,NaN,3.0,NaN,3.0,NaN,4315.0
Aldgate East,NaN,NaN,9.0,35.0,NaN,NaN,3.0,1.0,NaN,31.0,...,20.0,3.0,9.0,7.0,NaN,3.0,NaN,4.0,NaN,4707.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,11.0,34.0,35.0,NaN,NaN,NaN,NaN,35.0,...,26.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,5256.0
Woodgrange Park,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.0
Woodside Park,NaN,NaN,8.0,24.0,24.0,NaN,3.0,NaN,NaN,25.0,...,18.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,3570.0


In [92]:
print(CalcRSquared(cdatasub999["flows"],cdatasub999["uncosim"]))

0.19972301278106233


In [93]:
print(uncosim2.params[-1])

-0.15358439871289342


In [55]:
cdatasub99 = pd.read_csv("london_flows.csv")

In [87]:
#plus one before log distance because 0 exists in the dataset
cdatasub99["log_distance"] = np.log(cdatasub99['distance']+1)


x_variables = ["population", "jobs", "distance"]
log_x_vars = []
for x in x_variables:
    cdatasub99[f"log_{x}"] = np.log(cdatasub99[x]+1)
    log_x_vars.append(f"log_{x}")

In [88]:
formula = 'flows ~ log_population + log_jobs + log_distance'
uncosim1 = smf.glm(formula = formula, 
                  data=cdatasub99, 
                  family=sm.families.Poisson()).fit()
print(uncosim1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61474
Model:                            GLM   Df Residuals:                    61470
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.3935e+06
Date:                Mon, 09 May 2022   Deviance:                   2.6148e+06
Time:                        20:23:38   Pearson chi2:                 5.75e+06
No. Iterations:                    11                                         
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -7.6352      0.012   -659.

In [89]:
cdatasub99["uncosim"] = np.round(uncosim1.mu)
#here's the matrix
cdatasubmat_uncon = cdatasub99.pivot_table(values ="uncosim", index="station_origin", columns = "station_destination",
                                    aggfunc=np.sum, margins=True)
cdatasubmat_uncon

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,251.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,349.0
Acton Town,NaN,NaN,NaN,26.0,26.0,NaN,6.0,1.0,NaN,27.0,...,22.0,4.0,9.0,9.0,NaN,3.0,NaN,4.0,NaN,4393.0
Aldgate,NaN,NaN,8.0,NaN,41.0,NaN,NaN,1.0,NaN,33.0,...,18.0,NaN,8.0,7.0,NaN,3.0,NaN,3.0,NaN,4670.0
Aldgate East,NaN,NaN,8.0,44.0,NaN,NaN,3.0,1.0,NaN,35.0,...,19.0,3.0,9.0,7.0,NaN,4.0,NaN,4.0,NaN,5066.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,10.0,32.0,34.0,NaN,NaN,NaN,NaN,32.0,...,23.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,4867.0
Woodgrange Park,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0
Woodside Park,NaN,NaN,7.0,22.0,23.0,NaN,3.0,NaN,NaN,24.0,...,17.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,3360.0


In [62]:
formula = 'flows ~ station_origin + log_jobs + log_distance - 1'
#run a production constrained sim
prodSim = smf.glm(formula = formula, data=cdatasub99, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
print(prodSim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61474
Model:                            GLM   Df Residuals:                    61073
Model Family:                 Poisson   Df Model:                          400
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.1901e+06
Date:                Mon, 09 May 2022   Deviance:                   2.2080e+06
Time:                        18:55:11   Pearson chi2:                 3.83e+06
No. Iterations:                    26                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

In [63]:
cdatasub99["prodsim"] = np.round(prodSim.mu)
#here's the matrix
cdatasubmat_prod = cdatasub99.pivot_table(values ="prodsim", index="station_origin", columns = "station_destination",
                                    aggfunc=np.sum, margins=True)
cdatasubmat_prod

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,598.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1224.0
Acton Town,NaN,NaN,NaN,22.0,22.0,NaN,5.0,1.0,NaN,23.0,...,19.0,3.0,7.0,8.0,NaN,3.0,NaN,3.0,NaN,3742.0
Aldgate,NaN,NaN,5.0,NaN,26.0,NaN,NaN,1.0,NaN,21.0,...,11.0,NaN,5.0,4.0,NaN,2.0,NaN,2.0,NaN,2886.0
Aldgate East,NaN,NaN,5.0,28.0,NaN,NaN,2.0,1.0,NaN,22.0,...,11.0,2.0,5.0,4.0,NaN,2.0,NaN,2.0,NaN,3167.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,10.0,33.0,34.0,NaN,NaN,NaN,NaN,32.0,...,22.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,4869.0
Woodgrange Park,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,529.0
Woodside Park,NaN,NaN,7.0,21.0,21.0,NaN,3.0,NaN,NaN,22.0,...,15.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,3096.0


In [64]:
attr_form = 'flows ~ station_destination + log_population + log_distance-1'
#run a production constrained sim
attrSim = smf.glm(formula = attr_form, data=cdatasub99, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
print(attrSim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61474
Model:                            GLM   Df Residuals:                    61073
Model Family:                 Poisson   Df Model:                          400
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.2763e+06
Date:                Mon, 09 May 2022   Deviance:                   2.3804e+06
Time:                        18:57:43   Pearson chi2:                 4.61e+06
No. Iterations:                    26                                         
Covariance Type:            nonrobust                                         
                                                       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------

In [65]:
cdatasub99["attrsim"] = np.round(attrSim.mu)
#here's the matrix
cdatasubmat_attr = cdatasub99.pivot_table(values ="attrsim", index="station_origin", columns = "station_destination",
                                    aggfunc=np.sum, margins=True)
cdatasubmat_attr

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,436.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,633.0
Acton Town,NaN,NaN,NaN,23.0,22.0,NaN,5.0,1.0,NaN,23.0,...,24.0,3.0,7.0,8.0,NaN,2.0,NaN,3.0,NaN,4041.0
Aldgate,NaN,NaN,6.0,NaN,34.0,NaN,NaN,1.0,NaN,27.0,...,19.0,NaN,6.0,5.0,NaN,2.0,NaN,3.0,NaN,4294.0
Aldgate East,NaN,NaN,7.0,38.0,NaN,NaN,3.0,1.0,NaN,29.0,...,20.0,2.0,7.0,6.0,NaN,3.0,NaN,3.0,NaN,4652.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,8.0,29.0,29.0,NaN,NaN,NaN,NaN,27.0,...,24.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,4514.0
Woodgrange Park,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152.0
Woodside Park,NaN,NaN,6.0,20.0,19.0,NaN,3.0,NaN,NaN,20.0,...,18.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,3121.0


In [66]:
dbl_form = "flows ~ station_origin + station_destination + log_distance  -1"

doubSim = smf.glm(formula=dbl_form, data = cdatasub99, family = sm.families.Poisson()).fit()
print(doubSim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61474
Model:                            GLM   Df Residuals:                    60676
Model Family:                 Poisson   Df Model:                          797
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.1472e+06
Date:                Mon, 09 May 2022   Deviance:                   2.1222e+06
Time:                        19:00:26   Pearson chi2:                 3.60e+06
No. Iterations:                    27                                         
Covariance Type:            nonrobust                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [67]:
cdatasub99["doubsimfitted_log"] = np.round(doubSim.mu)
#here's the matrix
cdatasubmat_dou_log = cdatasub99.pivot_table(values ="doubsimfitted_log", index="station_origin", columns = "station_destination",
                                    aggfunc=np.sum, margins=True)
cdatasubmat_dou_log

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,601.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,1224.0
Acton Town,NaN,NaN,NaN,23.0,22.0,NaN,5.0,1.0,NaN,22.0,...,24.0,3.0,6.0,8.0,NaN,2.0,NaN,3.0,NaN,3747.0
Aldgate,NaN,NaN,4.0,NaN,25.0,NaN,NaN,1.0,NaN,20.0,...,13.0,NaN,4.0,4.0,NaN,2.0,NaN,2.0,NaN,2889.0
Aldgate East,NaN,NaN,5.0,29.0,NaN,NaN,2.0,1.0,NaN,21.0,...,14.0,2.0,5.0,4.0,NaN,2.0,NaN,2.0,NaN,3174.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,9.0,34.0,33.0,NaN,NaN,NaN,NaN,31.0,...,28.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,4866.0
Woodgrange Park,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,528.0
Woodside Park,NaN,NaN,6.0,21.0,20.0,NaN,3.0,NaN,NaN,21.0,...,19.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,3087.0


In [68]:
dbl_form_1 = "flows ~ station_origin + station_destination + distance  -1"

doubSim_1 = smf.glm(formula=dbl_form_1, data = cdatasub99, family = sm.families.Poisson()).fit()
print(doubSim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61474
Model:                            GLM   Df Residuals:                    60676
Model Family:                 Poisson   Df Model:                          797
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.1472e+06
Date:                Mon, 09 May 2022   Deviance:                   2.1222e+06
Time:                        19:03:03   Pearson chi2:                 3.60e+06
No. Iterations:                    27                                         
Covariance Type:            nonrobust                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [69]:
cdatasub99["doubsimfitted_nl"] = np.round(doubSim_1.mu)
#here's the matrix
cdatasubmat_dou_nl = cdatasub99.pivot_table(values ="doubsimfitted_nl", index="station_origin", columns = "station_destination",
                                    aggfunc=np.sum, margins=True)
cdatasubmat_dou_nl

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0,601.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1220.0
Acton Town,NaN,NaN,NaN,11.0,10.0,NaN,17.0,0.0,NaN,12.0,...,41.0,4.0,2.0,18.0,NaN,0.0,NaN,1.0,NaN,3745.0
Aldgate,NaN,NaN,1.0,NaN,32.0,NaN,NaN,0.0,NaN,23.0,...,7.0,NaN,3.0,2.0,NaN,1.0,NaN,1.0,NaN,2887.0
Aldgate East,NaN,NaN,2.0,38.0,NaN,NaN,0.0,0.0,NaN,25.0,...,7.0,1.0,3.0,2.0,NaN,1.0,NaN,1.0,NaN,3163.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,2.0,39.0,45.0,NaN,NaN,NaN,NaN,26.0,...,8.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,4867.0
Woodgrange Park,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,531.0
Woodside Park,NaN,NaN,2.0,17.0,16.0,NaN,1.0,NaN,NaN,26.0,...,10.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,3097.0


In [70]:
print(CalcRSquared(cdatasub99["flows"],cdatasub99["uncosim"]))

print(CalcRSquared(cdatasub99["flows"],cdatasub99["prodsim"]))
print(CalcRSquared(cdatasub99["flows"],cdatasub99["attrsim"]))

print(CalcRSquared(cdatasub99["flows"],cdatasub99["doubsimfitted_log"]))
print(CalcRSquared(cdatasub99["flows"],cdatasub99["doubsimfitted_nl"]))

0.16832508564133053
0.2196288010898645
0.2114024683299731
0.25747417495781744
0.47663287125851944


In [75]:
print(uncosim1.params[-1])
print(prodSim.params[-1])
print(attrSim.params[-1])
print(doubSim.params[-1])
print(doubSim_1.params[-1])

-0.2607217520080939
-0.2797312131365647
-0.2535948940759181
-0.2793723065593448
-0.00015184728753722825


In [94]:
index_iii2 = ['Unconstrained','Production_Constrained','Attraction_Constrained','Doubly_Constrained','Doubly_Constrained_neg_exp']
column_iii2 = ['R_Square', 'RMSE','Beta']
df_iii_2 = pd.DataFrame(index=index_iii2,columns=column_iii2)

In [95]:
df_iii_2['R_Square'][0]=CalcRSquared(cdatasub99["flows"],cdatasub99["uncosim"])
df_iii_2['R_Square'][1]=CalcRSquared(cdatasub99["flows"],cdatasub99["prodsim"])
df_iii_2['R_Square'][2]=CalcRSquared(cdatasub99["flows"],cdatasub99["attrsim"])
df_iii_2['R_Square'][3]=CalcRSquared(cdatasub99["flows"],cdatasub99["doubsimfitted_log"])
df_iii_2['R_Square'][4]=CalcRSquared(cdatasub99["flows"],cdatasub99["doubsimfitted_nl"])

df_iii_2['RMSE'][0]=CalcRMSE(cdatasub99["flows"],cdatasub99["uncosim"])
df_iii_2['RMSE'][1]=CalcRMSE(cdatasub99["flows"],cdatasub99["prodsim"])
df_iii_2['RMSE'][2]=CalcRMSE(cdatasub99["flows"],cdatasub99["attrsim"])
df_iii_2['RMSE'][3]=CalcRMSE(cdatasub99["flows"],cdatasub99["doubsimfitted_log"])
df_iii_2['RMSE'][4]=CalcRMSE(cdatasub99["flows"],cdatasub99["doubsimfitted_nl"])


df_iii_2['Beta'][0]=uncosim1.params[-1]
df_iii_2['Beta'][1]=prodSim.params[-1]
df_iii_2['Beta'][2]=attrSim.params[-1]
df_iii_2['Beta'][3]=doubSim.params[-1]
df_iii_2['Beta'][4]=doubSim_1.params[-1]


In [647]:
df_iii_2

,R_Square,RMSE,Beta
Unconstrained,0.168325,123.692,-0.260722
Production_Constrained,0.219629,120.407,-0.279731
Attraction_Constrained,0.211402,119.164,-0.253595
Doubly_Constrained,0.257474,115.905,-0.279372
Doubly_Constrained_neg_exp,0.476633,95.196,-0.000152


IV.1

In [163]:
cdatasub_4_a = pd.read_csv("london_flows.csv")

Production constrained model

In [164]:
#cdatasub_4_a["log_distance"] = np.log(cdatasub_4_a['distance']+1)
#cdatasub_4_a["log_Dj_jobs"] = np.log(cdatasub_4_a['jobs']+1)


#plus a constant before log distance because 0 exists in the dataset
x_variables = ["population", "jobs", "distance"]
log_x_vars = []
for x in x_variables:
    cdatasub_4_a[f"log_{x}"] = np.log(cdatasub_4_a[x]+0.01)
    log_x_vars.append(f"log_{x}")

In [165]:
formula = 'flows ~ station_origin + log_jobs + log_distance - 1'
#run a doubly constrained sim
prodSim_4_a = smf.glm(formula = formula, data=cdatasub_4_a, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
print(prodSim_4_a.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61474
Model:                            GLM   Df Residuals:                    61073
Model Family:                 Poisson   Df Model:                          400
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.2290e+06
Date:                Mon, 09 May 2022   Deviance:                   2.2857e+06
Time:                        22:27:03   Pearson chi2:                 4.29e+06
No. Iterations:                    26                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

In [166]:
cdatasub_4_a["prodsimfitted_4_a"] = np.round(prodSim_4_a.mu)
#here's the matrix
cdatasubmat_4_a = cdatasub_4_a.pivot_table(values ="prodsimfitted_4_a", index="station_origin", columns = "station_destination",
                                    aggfunc=np.sum, margins=True)
cdatasubmat_4_a

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,599.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1223.0
Acton Town,NaN,NaN,NaN,23.0,23.0,NaN,4.0,1.0,NaN,24.0,...,19.0,3.0,8.0,7.0,NaN,3.0,NaN,3.0,NaN,3743.0
Aldgate,NaN,NaN,5.0,NaN,22.0,NaN,NaN,1.0,NaN,20.0,...,12.0,NaN,6.0,4.0,NaN,2.0,NaN,2.0,NaN,2884.0
Aldgate East,NaN,NaN,6.0,24.0,NaN,NaN,2.0,1.0,NaN,21.0,...,13.0,2.0,6.0,5.0,NaN,2.0,NaN,2.0,NaN,3173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,10.0,32.0,33.0,NaN,NaN,NaN,NaN,32.0,...,24.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,4866.0
Woodgrange Park,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,530.0
Woodside Park,NaN,NaN,7.0,20.0,21.0,NaN,3.0,NaN,NaN,22.0,...,16.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,3090.0


In [167]:
CalcRSquared(cdatasub_4_a["flows"],cdatasub_4_a["prodsimfitted_4_a"])

0.2694876955425518

In [168]:
prodSim_4_a.params[-1]

-0.15556956050522797

In [169]:
O_i = pd.DataFrame(cdatasub_4_a.groupby(["station_origin"])["flows"].agg(np.sum))
O_i.rename(columns={"flows":"O_i"}, inplace = True)
cdatasub_4_a = cdatasub_4_a.merge(O_i, on = "station_origin", how = "left" )

D_j = pd.DataFrame(cdatasub_4_a.groupby(["station_destination"])["flows"].agg(np.sum))
D_j.rename(columns={"flows":"D_j"}, inplace = True)
cdatasub_4_a = cdatasub_4_a.merge(D_j, on = "station_destination", how = "left" )

In [170]:
coefs = pd.DataFrame(prodSim_4_a.params)
coefs.reset_index(inplace=True)
coefs.rename(columns = {0:"alpha_i", "index":"coef"}, inplace = True)
to_repl = ["(station_origin)", "\[", "\]"]
for x in to_repl:
    coefs["coef"] = coefs["coef"].str.replace(x, "")
#then once you have done this you can join them back into the dataframes
cdatasub_4_a = cdatasub_4_a.merge(coefs, left_on="station_origin", right_on="coef", how = "left")
cdatasub_4_a.drop(columns = ["coef"], inplace = True)
#check this has worked
cdatasub_4_a.head()

/tmp/ipykernel_3994/2820640461.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  coefs["coef"] = coefs["coef"].str.replace(x, "")


,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,prodsimfitted_4_a,O_i,D_j,alpha_i
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,6.395278,11.271478,9.003505,131.0,599,78549,-2.857836
1,Abbey Road,Beckton,1,599,442,8510.121774,6.395278,6.091333,9.049013,2.0,599,442,-2.857836
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.395278,6.499802,8.236277,3.0,599,665,-2.857836
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395278,10.981421,8.534350,111.0,599,58772,-2.857836
4,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395278,9.643940,7.709278,43.0,599,15428,-2.857836


In [171]:
alpha_i = prodSim_4_a.params[0:399]
gamma_4a = prodSim_4_a.params[-2]
beta_4a = prodSim_4_a.params[-1]
print(gamma_4a)
print(beta_4a)

0.8102500363535149
-0.15556956050522797


Reduce job to 50% at Canary Wharf

In [186]:
cdatasub_4_a2 = cdatasub_4_a.copy()
cdatasub_4_a_final = cdatasub_4_a.copy()
cdatasub_4_a2 = cdatasub_4_a2[['station_destination', 'jobs']]

In [187]:
cdatasub_4_a2.drop_duplicates(subset='station_destination', inplace=True)
cdatasub_4_a2.iloc[[3],[1]] = cdatasub_4_a2.iloc[[3],[1]] * 0.5

In [188]:
cdatasub_4_a2[cdatasub_4_a2['station_destination']=='Canary Wharf']

,station_destination,jobs
3,Canary Wharf,29386.0


In [189]:
cdatasub_4_a_final = pd.merge(cdatasub_4_a_final, cdatasub_4_a2,left_on='station_destination', right_on='station_destination', how='left')

In [197]:
cdatasub_4_a_final

,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,prodsimfitted_4_a,O_i,D_j,alpha_i,jobs_CW0.5,prodsimest2,Ai1,A_i,prodsimest3,A_i2,prodsimest4
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,6.395278,11.271478,9.003505,131.0,599,78549,-2.857836,78549.0,2155.0,37548.417547,0.000007,164.0,0.000008,180.0
1,Abbey Road,Beckton,1,599,442,8510.121774,6.395278,6.091333,9.049013,2.0,599,442,-2.857836,442.0,33.0,568.632753,0.000007,2.0,0.000008,3.0
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.395278,6.499802,8.236277,3.0,599,665,-2.857836,665.0,40.0,697.684015,0.000007,3.0,0.000008,3.0
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395278,10.981421,8.534350,111.0,599,58772,-2.857836,29386.0,903.0,15736.826368,0.000007,121.0,0.000008,75.0
4,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395278,9.643940,7.709278,43.0,599,15428,-2.857836,15428.0,471.0,8211.825519,0.000007,36.0,0.000008,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61469,Woolwich Arsenal,Tower Gateway,127,7892,3342,13401.795549,8.973606,8.114328,9.503145,140.0,7892,3342,-0.152205,3342.0,2700.0,3143.310497,0.000006,151.0,0.000007,164.0
61470,Woolwich Arsenal,West Ham,608,7892,5487,8701.454361,8.973606,8.610139,9.071247,224.0,7892,5487,-0.152205,5487.0,3772.0,4392.146925,0.000006,211.0,0.000007,230.0
61471,Woolwich Arsenal,West India Quay,6,7892,400,9536.720451,8.973606,5.991490,9.162906,26.0,7892,400,-0.152205,400.0,458.0,533.817202,0.000006,26.0,0.000007,28.0
61472,Woolwich Arsenal,West Silvertown,81,7892,893,5355.248554,8.973606,6.794598,8.585834,56.0,7892,893,-0.152205,893.0,803.0,935.431759,0.000006,45.0,0.000007,49.0


In [190]:
cdatasub_4_a_final.rename(columns = {'jobs_x':'jobs','jobs_y':'jobs_CW0.5'}, inplace = True)

In [191]:
cdatasub_4_a_final["prodsimest2"] = np.exp(cdatasub_4_a_final["alpha_i"] +
                                           gamma_4a*np.log(cdatasub_4_a_final["jobs_CW0.5"]) - 
                                           beta_4a*cdatasub_4_a_final["log_distance"])

cdatasub_4_a_final["prodsimest2"] = round(cdatasub_4_a_final["prodsimest2"],0)
#now we can convert the pivot table into a matrix
cdatasubmat_4_a_final = cdatasub_4_a_final.pivot_table(values ="prodsimest2", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)
cdatasubmat_4_a_final

/opt/conda/envs/sds2021/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,214.0,7174.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,54.0,NaN,NaN,23833.0
Acton Town,NaN,NaN,NaN,470.0,479.0,NaN,58.0,33.0,NaN,480.0,...,391.0,57.0,174.0,111.0,NaN,74.0,NaN,75.0,NaN,71762.0
Aldgate,NaN,NaN,111.0,NaN,214.0,NaN,NaN,23.0,NaN,251.0,...,266.0,NaN,104.0,81.0,NaN,44.0,NaN,46.0,NaN,41906.0
Aldgate East,NaN,NaN,121.0,229.0,NaN,NaN,52.0,25.0,NaN,275.0,...,289.0,42.0,114.0,88.0,NaN,47.0,NaN,51.0,NaN,46424.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,258.0,644.0,644.0,NaN,NaN,NaN,NaN,679.0,...,610.0,NaN,239.0,NaN,NaN,NaN,NaN,NaN,NaN,102761.0
Woodgrange Park,NaN,184.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10506.0
Woodside Park,NaN,NaN,163.0,426.0,434.0,NaN,69.0,NaN,NaN,428.0,...,387.0,NaN,155.0,NaN,NaN,NaN,NaN,NaN,NaN,63856.0


In [192]:
Dj2_gamma = cdatasub_4_a_final["jobs"]**gamma_4a
dist_beta = cdatasub_4_a_final["distance"]**-beta_4a


cdatasub_4_a_final["Ai1"] = Dj2_gamma * dist_beta


A_i = pd.DataFrame(cdatasub_4_a_final.groupby(["station_origin"])["Ai1"].agg(np.sum))


A_i["Ai1"] = 1/A_i["Ai1"]
A_i.rename(columns={"Ai1":"A_i"}, inplace=True)


cdatasub_4_a_final = cdatasub_4_a_final.merge(A_i, left_on="station_origin", right_index=True, how="left")

In [193]:
#to check everything works, recreate the original estimates
cdatasub_4_a_final["prodsimest3"] = cdatasub_4_a_final["A_i"]*cdatasub_4_a_final["O_i"]*Dj2_gamma*dist_beta
#round
cdatasub_4_a_final["prodsimest3"] = round(cdatasub_4_a_final["prodsimest3"])
#check
cdatasub_4_a_final[["prodsimfitted_4_a", "prodsimest3"]]

,prodsimfitted_4_a,prodsimest3
0,131.0,164.0
1,2.0,2.0
2,3.0,3.0
3,111.0,121.0
4,43.0,36.0
...,...,...
61469,140.0,151.0
61470,224.0,211.0
61471,26.0,26.0
61472,56.0,45.0


In [194]:
#calculate some new wj^alpha and d_ij^beta values
Dj3_gamma = cdatasub_4_a_final["jobs_CW0.5"]**gamma_4a

#calcualte the first stage of the Ai values
cdatasub_4_a_final["Ai1"] = Dj3_gamma * dist_beta

#now do the sum over all js bit
A_i = pd.DataFrame(cdatasub_4_a_final.groupby(["station_origin"])["Ai1"].agg(np.sum))

#now divide into 1
A_i["Ai1"] = 1/A_i["Ai1"]
A_i.rename(columns={"Ai1":"A_i2"}, inplace=True)

#and write the A_i values back into the dataframe
cdatasub_4_a_final = cdatasub_4_a_final.merge(A_i, left_on="station_origin", right_index=True, how="left")

In [195]:
#to check everything works, recreate the original estimates
cdatasub_4_a_final["prodsimest4"] = cdatasub_4_a_final["A_i2"]*cdatasub_4_a_final["O_i"]*Dj3_gamma*dist_beta
#round
cdatasub_4_a_final["prodsimest4"] = round(cdatasub_4_a_final["prodsimest4"])

In [196]:
cdatasubmat5 = cdatasub_4_a_final.pivot_table(values ="prodsimest4", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)
cdatasubmat5

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,597.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,1223.0
Acton Town,NaN,NaN,NaN,25.0,25.0,NaN,3.0,2.0,NaN,25.0,...,20.0,3.0,9.0,6.0,NaN,4.0,NaN,4.0,NaN,3751.0
Aldgate,NaN,NaN,8.0,NaN,15.0,NaN,NaN,2.0,NaN,17.0,...,18.0,NaN,7.0,6.0,NaN,3.0,NaN,3.0,NaN,2884.0
Aldgate East,NaN,NaN,8.0,16.0,NaN,NaN,4.0,2.0,NaN,19.0,...,20.0,3.0,8.0,6.0,NaN,3.0,NaN,3.0,NaN,3178.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,12.0,30.0,31.0,NaN,NaN,NaN,NaN,32.0,...,29.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,4868.0
Woodgrange Park,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,529.0
Woodside Park,NaN,NaN,8.0,21.0,21.0,NaN,3.0,NaN,NaN,21.0,...,19.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,3099.0


In [633]:
cdatasubmat_4_a.fillna(0)

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,599.0
Acton Central,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1223.0
Acton Town,0.0,0.0,0.0,23.0,23.0,0.0,4.0,1.0,0.0,24.0,...,19.0,3.0,8.0,7.0,0.0,3.0,0.0,3.0,0.0,3743.0
Aldgate,0.0,0.0,5.0,0.0,22.0,0.0,0.0,1.0,0.0,20.0,...,12.0,0.0,6.0,4.0,0.0,2.0,0.0,2.0,0.0,2884.0
Aldgate East,0.0,0.0,6.0,24.0,0.0,0.0,2.0,1.0,0.0,21.0,...,13.0,2.0,6.0,5.0,0.0,2.0,0.0,2.0,0.0,3173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,0.0,0.0,10.0,32.0,33.0,0.0,0.0,0.0,0.0,32.0,...,24.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,4866.0
Woodgrange Park,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,530.0
Woodside Park,0.0,0.0,7.0,20.0,21.0,0.0,3.0,0.0,0.0,22.0,...,16.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,3090.0


In [634]:
cdatasubmat5.fillna(0)

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,597.0
Acton Central,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1223.0
Acton Town,0.0,0.0,0.0,25.0,25.0,0.0,3.0,2.0,0.0,25.0,...,20.0,3.0,9.0,6.0,0.0,4.0,0.0,4.0,0.0,3751.0
Aldgate,0.0,0.0,8.0,0.0,15.0,0.0,0.0,2.0,0.0,17.0,...,18.0,0.0,7.0,6.0,0.0,3.0,0.0,3.0,0.0,2884.0
Aldgate East,0.0,0.0,8.0,16.0,0.0,0.0,4.0,2.0,0.0,19.0,...,20.0,3.0,8.0,6.0,0.0,3.0,0.0,3.0,0.0,3178.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,0.0,0.0,12.0,30.0,31.0,0.0,0.0,0.0,0.0,32.0,...,29.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,4868.0
Woodgrange Park,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,529.0
Woodside Park,0.0,0.0,8.0,21.0,21.0,0.0,3.0,0.0,0.0,21.0,...,19.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,3099.0


In [637]:
cdatasubmat_net = (cdatasubmat5-cdatasubmat_4_a).fillna(0)

In [639]:
cdatasubmat_net['Canary Wharf']

station_origin
Abbey Road            -36.0
Acton Central           0.0
Acton Town            -40.0
Aldgate               -35.0
Aldgate East          -45.0
                     ...   
Woodford              -79.0
Woodgrange Park         0.0
Woodside Park         -38.0
Woolwich Arsenal     -590.0
All                -24614.0
Name: Canary Wharf, Length: 400, dtype: float64

In [678]:
cdatasub_4_a_final["flow_diff"] = (cdatasub_4_a_final["prodsimest4"] - cdatasub_4_a_final["prodsimfitted_4_a"])


In [683]:
flow_net=cdatasub_4_a_final.sort_values(by='flow_diff',ascending=False)

In [688]:
flow_net = flow_net[["station_origin","station_destination","flow_diff"]]

In [704]:
flow_net.tail(10)

,station_origin,station_destination,flow_diff
49948,Stratford,Canary Wharf,-979.0
18940,Finsbury Park,Finsbury Park,-1080.0
22242,Hammersmith,Hammersmith,-1124.0
17552,Euston,Euston,-1434.0
39328,Paddington,Paddington,-2272.0
8847,Canary Wharf,Canary Wharf,-2989.0
32755,London Bridge,London Bridge,-4022.0
54895,Victoria,Victoria,-5053.0
32472,Liverpool Street,Liverpool Street,-6251.0
56180,Waterloo,Waterloo,-6934.0


In [694]:
flow_net[flow_net['station_destination'] == 'Canary Wharf'].sort_values(by='flow_diff',ascending=True)

,station_origin,station_destination,flow_diff
8847,Canary Wharf,Canary Wharf,-2989.0
49948,Stratford,Canary Wharf,-979.0
31708,Lewisham,Canary Wharf,-687.0
61442,Woolwich Arsenal,Canary Wharf,-590.0
8542,Canada Water,Canary Wharf,-567.0
...,...,...,...
12711,Covent Garden,Canary Wharf,-4.0
51008,Temple,Canary Wharf,-4.0
42816,Regent's Park,Canary Wharf,-3.0
20350,Goodge Street,Canary Wharf,-3.0


IV.2

In [247]:

# Here is the entropy maximising approach for a known beta.
# Plug in the required values in this function to solve.

def balance_doubly_constrained(pd, orig_field, dest_field, Oi_field, Dj_field, cij_field, beta, 
                               cost_function, Ai_name = "Ai_new", Bj_name = "Bj_new", converge=0.001):
    # Define some variables
    Oi = pd[[orig_field, Oi_field]]
    Dj = pd[[dest_field,Dj_field]]    
    if cost_function.lower() in ['power','pow']:
        beta_cij = np.exp(beta * np.log(pd[cij_field]))
    elif cost_function.lower() in ['exponential','exp']:
        beta_cij = np.exp(beta * pd[cij_field])
    else:
        return "Cost function not specified properly, use 'exp' or 'pow'"
    
    # Create some helper variables
    cnvg = 1
    iteration = 0
    # Now iteratively rebalance the Ai and Bj terms until convergence
    while cnvg > converge:
        if iteration == 0:
            # This first condition sets starting values for Ai and Bj
            # NB sets starting value of Ai assuming Bj is a vector of 1s.
            # We've already established beta_cij with the appropriate cost function, so...
            Oi = Oi.assign(Ai = Dj[Dj_field] * beta_cij)
            # Aggregate Ai and take inverse
            Ai = 1.0/Oi.groupby(orig_field)['Ai'].sum().to_frame()
            # Merge new Ais 
            Oi = Oi.merge(Ai,left_on = orig_field, right_index = True, suffixes = ('','_old'))
            # Drop the temporary Ai field we created, leaving Ai_old
            Oi.drop('Ai', axis=1, inplace=True)
            
            # Now set up Bjs using starting values of Ai
            Dj = Dj.assign(Bj = Oi['Ai_old'] * Oi[Oi_field] * beta_cij)
            # Aggregate Bj and take inverse
            Bj = 1.0/Dj.groupby(dest_field)['Bj'].sum().to_frame()
            # Merge new Bjs
            Dj = Dj.merge(Bj,left_on = dest_field, right_index = True, suffixes = ('','_old'))
            # Drop the temporary Bj field we created, leaving Bj_old
            Dj.drop('Bj', axis=1, inplace=True)
            
            # Increment loop
            iteration += 1
        else:
            # This bit is the iterated bit of the loop which refines the values of Ai and Bj
            # First Ai
            Oi['Ai'] = Dj['Bj_old'] * Dj[Dj_field] * beta_cij
            # Aggregate Ai and take inverse
            Ai = 1.0/Oi.groupby(orig_field)['Ai'].sum().to_frame()
            # Drop temporary Ai
            Oi.drop('Ai', axis=1, inplace=True)
            # Merge new Ais 
            Oi = Oi.merge(Ai,left_on = orig_field, right_index = True)
            # Calculate the difference between old and new Ais
            Oi['diff'] = np.absolute((Oi['Ai_old'] - Oi['Ai'])/Oi['Ai_old'])
            # Set new Ais to Ai_old
            Oi['Ai_old'] = Oi['Ai']
            # Drop the temporary Ai field we created, leaving Ai_old
            Oi.drop('Ai', axis=1, inplace=True)
            
            # Then Bj
            Dj['Bj'] = Oi['Ai_old'] * Oi[Oi_field] * beta_cij
            # Aggregate Bj and take inverse
            Bj = 1.0/Dj.groupby(dest_field)['Bj'].sum().to_frame()
            # Drop temporary Bj
            Dj.drop('Bj', axis=1, inplace=True)
            # Merge new Bjs
            Dj = Dj.merge(Bj,left_on = dest_field, right_index = True)
            # Calculate the difference between old and new Bjs
            Dj['diff'] = np.absolute((Dj['Bj_old'] - Dj['Bj'])/Dj['Bj_old'])
            # Set new Bjs to Bj_old
            Dj['Bj_old'] = Dj['Bj']
            # Drop the temporary Bj field we created, leaving Bj_old
            Dj.drop('Bj', axis=1, inplace=True)
            
            # Assign higher sum difference from Ai or Bj to cnvg
            cnvg = np.maximum(Oi['diff'].sum(),Dj['diff'].sum())
            
            # Print and increment loop
            print("Iteration:", iteration)
            iteration += 1

    # When the while loop finishes add the computed Ai_old and Bj_old to the dataframe and return
    pd[Ai_name] = Oi['Ai_old']
    pd[Bj_name] = Dj['Bj_old']
    return pd

In [270]:
cdatasub_4b = pd.read_csv("london_flows.csv")

In [271]:
x_variables = ["population", "jobs", "distance"]
log_x_vars = []
for x in x_variables:
    cdatasub_4b[f"log_{x}"] = np.log(cdatasub_4b[x]+0.01)
    log_x_vars.append(f"log_{x}")

In [272]:
dbl_form_4b = 'flows ~ station_destination + station_origin + log_distance - 1'
#run a doubly constrained sim
doubSim_4b = smf.glm(formula = dbl_form_4b, data=cdatasub_4b, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
print(doubSim_4b.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61474
Model:                            GLM   Df Residuals:                    60676
Model Family:                 Poisson   Df Model:                          797
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.1831e+06
Date:                Tue, 10 May 2022   Deviance:                   2.1940e+06
Time:                        02:09:07   Pearson chi2:                 4.15e+06
No. Iterations:                    27                                         
Covariance Type:            nonrobust                                         
                                                       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------

In [273]:
#get the estimates
cdatasub_4b["doubsimfitted"] = np.round(doubSim_4b.mu)
#here's the matrix
cdatasubmat_4b_1 = cdatasub_4b.pivot_table(values ="doubsimfitted", index="station_origin", columns = "station_destination",
                                    aggfunc=np.sum, margins=True)
cdatasubmat_4b_1

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.0,600.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,1226.0
Acton Town,NaN,NaN,NaN,24.0,23.0,NaN,4.0,1.0,NaN,23.0,...,22.0,3.0,6.0,7.0,NaN,2.0,NaN,3.0,NaN,3751.0
Aldgate,NaN,NaN,5.0,NaN,22.0,NaN,NaN,1.0,NaN,19.0,...,14.0,NaN,5.0,4.0,NaN,2.0,NaN,2.0,NaN,2886.0
Aldgate East,NaN,NaN,5.0,25.0,NaN,NaN,2.0,1.0,NaN,21.0,...,15.0,2.0,5.0,4.0,NaN,2.0,NaN,2.0,NaN,3173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,9.0,33.0,33.0,NaN,NaN,NaN,NaN,31.0,...,27.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,4865.0
Woodgrange Park,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,532.0
Woodside Park,NaN,NaN,6.0,21.0,20.0,NaN,3.0,NaN,NaN,21.0,...,18.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,3087.0


In [274]:
CalcRSquared(cdatasub_4b["flows"],cdatasub_4b["doubsimfitted"])

0.3099326527078569

In [454]:
dbl_form_4b_o = 'flows ~ station_origin + station_destination + log_distance - 1'
#run a doubly constrained sim
doubSim_4b_o = smf.glm(formula = dbl_form_4b_o, data=cdatasub_4b, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
print(doubSim_4b_o.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61474
Model:                            GLM   Df Residuals:                    60676
Model Family:                 Poisson   Df Model:                          797
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.1831e+06
Date:                Tue, 10 May 2022   Deviance:                   2.1940e+06
Time:                        03:33:22   Pearson chi2:                 4.15e+06
No. Iterations:                    27                                         
Covariance Type:            nonrobust                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [606]:
cdatasub_4b_final = cdatasub_4b.copy()

In [607]:
O_i_4b = pd.DataFrame(cdatasub_4b_final.groupby(["station_origin"])["flows"].agg(np.sum))
O_i_4b.rename(columns={"flows":"O_i_4b"}, inplace = True)
cdatasub_4b_final = cdatasub_4b_final.merge(O_i_4b, on = "station_origin", how = "left" )

D_j_4b = pd.DataFrame(cdatasub_4b_final.groupby(["station_destination"])["flows"].agg(np.sum))
D_j_4b.rename(columns={"flows":"D_j_4b"}, inplace = True)
cdatasub_4b_final = cdatasub_4b_final.merge(D_j_4b, on = "station_destination", how = "left" )


In [608]:
#We can do this by pulling out the parameter values
coefs_d = pd.DataFrame(doubSim_4b.params[0:399])
coefs_d.reset_index(inplace=True)
coefs_d.rename(columns = {0:"gamma_j", "index":"coef_d"}, inplace = True)
to_repl = ["(station_destination)", "\[", "\]"]
for x in to_repl:
    coefs_d["coef_d"] = coefs_d["coef_d"].str.replace(x, "")
#then once you have done this you can join them back into the dataframes
cdatasub_4b_final = cdatasub_4b_final.merge(coefs_d, left_on="station_destination", right_on="coef_d", how = "left")
cdatasub_4b_final.drop(columns = ["coef_d"], inplace = True)
#check this has worked
cdatasub_4b_final.head()

/tmp/ipykernel_3994/2749065237.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  coefs_d["coef_d"] = coefs_d["coef_d"].str.replace(x, "")


,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,doubsimfitted,O_i_4b,D_j_4b,gamma_j
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,6.395278,11.271478,9.003505,117.0,599,78549,6.111476
1,Abbey Road,Beckton,1,599,442,8510.121774,6.395278,6.091333,9.049013,3.0,599,442,2.444511
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.395278,6.499802,8.236277,4.0,599,665,2.738600
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395278,10.981421,8.534350,91.0,599,58772,5.789534
4,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395278,9.643940,7.709278,29.0,599,15428,4.525742


In [609]:
#We can do this by pulling out the parameter values
coefs_o = pd.DataFrame(doubSim_4b_o.params[0:399])
coefs_o.reset_index(inplace=True)
coefs_o.rename(columns = {0:"alpha_i", "index":"coef_o"}, inplace = True)
to_repl = ["(station_origin)", "\[", "\]"]
for x in to_repl:
    coefs_o["coef_o"] = coefs_o["coef_o"].str.replace(x, "")

#then once you have done this you can join them back into the dataframes
cdatasub_4b_final = cdatasub_4b_final.merge(coefs_o, left_on="station_origin", right_on="coef_o", how = "left")
cdatasub_4b_final.drop(columns = ["coef_o"], inplace = True)
#check this has worked
cdatasub_4b_final.head()


/tmp/ipykernel_3994/1914072414.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  coefs_o["coef_o"] = coefs_o["coef_o"].str.replace(x, "")


,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,doubsimfitted,O_i_4b,D_j_4b,gamma_j,alpha_i
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,6.395278,11.271478,9.003505,117.0,599,78549,6.111476,2.212549
1,Abbey Road,Beckton,1,599,442,8510.121774,6.395278,6.091333,9.049013,3.0,599,442,2.444511,2.212549
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.395278,6.499802,8.236277,4.0,599,665,2.738600,2.212549
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395278,10.981421,8.534350,91.0,599,58772,5.789534,2.212549
4,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395278,9.643940,7.709278,29.0,599,15428,4.525742,2.212549


In [610]:
beta_4 = doubSim_4b.params[-1]

In [650]:
beta_4b_hundred = beta_4 * 0.01
beta_4b_tenth = beta_4 * 0.1

In [651]:
print(beta_4)
print(beta_4b_tenth)
print(beta_4b_hundred)

-0.15024454560585385
-0.015024454560585385
-0.0015024454560585384


In [613]:
cdatasub_4b_final["doubsim_old"] = np.exp(cdatasub_4b_final["alpha_i"]+cdatasub_4b_final["gamma_j"] 
                                 - beta_4*cdatasub_4b_final["log_distance"])
#or you could do it the easy way like we did last week with the fitted column (See previous practical)
cdatasub_4b_final.head(10)

cdatasub_4b_final["doubsim_old"] = round(cdatasub_4b_final["doubsim_old"],0)

In [614]:
#tenth!!!

cdatasub_4b_final["doubsim2_t"] = np.exp(cdatasub_4b_final["alpha_i"]+cdatasub_4b_final["gamma_j"] - beta_4b_tenth*cdatasub_4b_final["log_distance"])

cdatasub_4b_final["doubsim2_t"] = round(cdatasub_4b_final["doubsim2_t"],0)
#now we can convert the pivot table into a matrix
cdatasubmat4b2 = cdatasub_4b_final.pivot_table(values ="doubsim2_t", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)
cdatasubmat4b2

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1533.0,22029.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,234.0,NaN,NaN,54222.0
Acton Town,NaN,NaN,NaN,1084.0,1048.0,NaN,139.0,59.0,NaN,1042.0,...,996.0,118.0,309.0,263.0,NaN,121.0,NaN,135.0,NaN,165157.0
Aldgate,NaN,NaN,217.0,NaN,668.0,NaN,NaN,39.0,NaN,674.0,...,660.0,NaN,202.0,176.0,NaN,79.0,NaN,88.0,NaN,108517.0
Aldgate East,NaN,NaN,236.0,752.0,NaN,NaN,103.0,43.0,NaN,735.0,...,720.0,85.0,221.0,192.0,NaN,86.0,NaN,96.0,NaN,119861.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,458.0,1499.0,1448.0,NaN,NaN,NaN,NaN,1446.0,...,1395.0,NaN,427.0,NaN,NaN,NaN,NaN,NaN,NaN,226233.0
Woodgrange Park,NaN,477.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23168.0
Woodside Park,NaN,NaN,296.0,973.0,941.0,NaN,129.0,NaN,NaN,934.0,...,902.0,NaN,277.0,NaN,NaN,NaN,NaN,NaN,NaN,141937.0


In [615]:
#hundred!!!

cdatasub_4b_final["doubsim2_h"] = np.exp(cdatasub_4b_final["alpha_i"]+cdatasub_4b_final["gamma_j"] - beta_4b_hundred*cdatasub_4b_final["log_distance"])

cdatasub_4b_final["doubsim2_h"] = round(cdatasub_4b_final["doubsim2_h"],0)
#now we can convert the pivot table into a matrix
cdatasubmat4b2 = cdatasub_4b_final.pivot_table(values ="doubsim2_h", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)
cdatasubmat4b2

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1355.0,19650.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,204.0,NaN,NaN,47634.0
Acton Town,NaN,NaN,NaN,950.0,919.0,NaN,124.0,51.0,NaN,914.0,...,874.0,104.0,270.0,234.0,NaN,105.0,NaN,117.0,NaN,145176.0
Aldgate,NaN,NaN,190.0,NaN,605.0,NaN,NaN,34.0,NaN,603.0,...,578.0,NaN,178.0,155.0,NaN,69.0,NaN,78.0,NaN,96590.0
Aldgate East,NaN,NaN,207.0,682.0,NaN,NaN,90.0,37.0,NaN,657.0,...,630.0,75.0,194.0,169.0,NaN,75.0,NaN,84.0,NaN,106648.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,398.0,1316.0,1272.0,NaN,NaN,NaN,NaN,1266.0,...,1211.0,NaN,373.0,NaN,NaN,NaN,NaN,NaN,NaN,198023.0
Woodgrange Park,NaN,416.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20373.0
Woodside Park,NaN,NaN,258.0,853.0,825.0,NaN,112.0,NaN,NaN,821.0,...,785.0,NaN,242.0,NaN,NaN,NaN,NaN,NaN,NaN,124384.0


In [616]:
# Use the beta we got from the inverse power model
beta_4b = doubSim_4b.params[-1]
# Get the balancing factors.
cdatasub_4b_final = balance_doubly_constrained(cdatasub_4b_final,'station_origin','station_destination','O_i_4b','D_j_4b','distance',-beta_4b,'power')

# Now predict the model again using the new Ai and Dj fields.
cdatasub_4b_final['SIM_est_pow'] = np.round(cdatasub_4b_final['O_i_4b'] * cdatasub_4b_final['Ai_new'] * cdatasub_4b_final['D_j_4b'] * cdatasub_4b_final['Bj_new'] * 
                                   np.exp(np.log(cdatasub_4b_final['distance'])*-beta_4b))
# Check out the matrix
pivot_old = pd.pivot_table(cdatasub_4b_final,values='SIM_est_pow',index ='station_origin',columns='station_destination',fill_value=0,aggfunc=sum,margins=True)

/opt/conda/envs/sds2021/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21


In [617]:
cdatasub_4b_final["SIM_est_pow_old"] = (cdatasub_4b_final["Ai_new"]*cdatasub_4b_final["Bj_new"]
                                        *cdatasub_4b_final["O_i_4b"]*cdatasub_4b_final["D_j_4b"]*(cdatasub_4b_final["distance"]**-beta_4b)
                                        )
cdatasub_4b_final["SIM_est_pow_old"] = round(cdatasub_4b_final["SIM_est_pow_old"])

In [618]:
# Use the beta we got from the inverse power model
beta_4b_h = beta_4b_hundred
# Get the balancing factors.
cdatasub_4b_final = balance_doubly_constrained(cdatasub_4b_final,'station_origin','station_destination','O_i_4b','D_j_4b','distance',-beta_4b_h,'power')

# Now predict the model again using the new Ai and Dj fields.
cdatasub_4b_final['SIM_est_pow_h'] = np.round(cdatasub_4b_final['O_i_4b'] * cdatasub_4b_final['Ai_new'] * cdatasub_4b_final['D_j_4b'] * cdatasub_4b_final['Bj_new'] * 
                                   np.exp(np.log(cdatasub_4b_final['distance'])*-beta_4b_h))
# Check out the matrix
pivot_hundred =pd.pivot_table(cdatasub_4b_final,values='SIM_est_pow_h',index ='station_origin',columns='station_destination',fill_value=0,aggfunc=sum,margins=True)

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21


In [619]:
cdatasub_4b_final["SIM_est_pow_hundred"] = (cdatasub_4b_final["Ai_new"]*cdatasub_4b_final["Bj_new"]
                                        *cdatasub_4b_final["O_i_4b"]*cdatasub_4b_final["D_j_4b"]*(cdatasub_4b_final["distance"]**-beta_4b_hundred)
                                        )
cdatasub_4b_final["SIM_est_pow_hundred"] = round(cdatasub_4b_final["SIM_est_pow_hundred"])

In [620]:
cdatasub_4b_final[["SIM_est_pow_old", "SIM_est_pow_hundred"]]

,SIM_est_pow_old,SIM_est_pow_hundred
0,138.0,126.0
1,3.0,3.0
2,5.0,5.0
3,91.0,93.0
4,21.0,24.0
...,...,...
61469,293.0,287.0
61470,104.0,113.0
61471,37.0,35.0
61472,73.0,79.0


In [621]:
# Use the beta we got from the inverse power model
beta_4b_t = beta_4b_tenth
# Get the balancing factors.
cdatasub_4b_final = balance_doubly_constrained(cdatasub_4b_final,'station_origin','station_destination','O_i_4b','D_j_4b','distance',-beta_4b_t,'power')

# Now predict the model again using the new Ai and Dj fields.
cdatasub_4b_final['SIM_est_pow_t'] = np.round(cdatasub_4b_final['O_i_4b'] * cdatasub_4b_final['Ai_new'] * cdatasub_4b_final['D_j_4b'] * cdatasub_4b_final['Bj_new'] * 
                                   np.exp(np.log(cdatasub_4b_final['distance'])*-beta_4b_t))
# Check out the matrix
pivot_tenth = pd.pivot_table(cdatasub_4b_final,values='SIM_est_pow_t',index ='station_origin',columns='station_destination',fill_value=0,aggfunc=sum,margins=True)

/opt/conda/envs/sds2021/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21


In [622]:
cdatasub_4b_final["SIM_est_pow_tenth"] = (cdatasub_4b_final["Ai_new"]*cdatasub_4b_final["Bj_new"]
                                        *cdatasub_4b_final["O_i_4b"]*cdatasub_4b_final["D_j_4b"]*(cdatasub_4b_final["distance"]**-beta_4b_tenth)
                                        )
cdatasub_4b_final["SIM_est_pow_tenth"] = round(cdatasub_4b_final["SIM_est_pow_tenth"])

In [623]:
cdatasub_4b_final[["SIM_est_pow_old", "SIM_est_pow_tenth", "SIM_est_pow_hundred"]]

,SIM_est_pow_old,SIM_est_pow_tenth,SIM_est_pow_hundred
0,138.0,127.0,126.0
1,3.0,3.0,3.0
2,5.0,5.0,5.0
3,91.0,93.0,93.0
4,21.0,24.0,24.0
...,...,...,...
61469,293.0,287.0,287.0
61470,104.0,112.0,113.0
61471,37.0,35.0,35.0
61472,73.0,79.0,79.0


In [631]:
matrix_old_tenth = pivot_old - pivot_tenth

In [632]:
matrix_old_tenth.to_csv('matrix_old_tenth.csv')

In [706]:
cdatasub_4b_final

,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,doubsimfitted,...,doubsim2_t,doubsim2_h,Ai_new,Bj_new,SIM_est_pow,SIM_est_pow_old,SIM_est_pow_h,SIM_est_pow_hundred,SIM_est_pow_t,SIM_est_pow_tenth
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,6.395278,11.271478,9.003505,117.0,...,4719.0,4178.0,0.000003,0.849987,138.0,138.0,126.0,126.0,127.0,127.0
1,Abbey Road,Beckton,1,599,442,8510.121774,6.395278,6.091333,9.049013,3.0,...,121.0,107.0,0.000003,3.517759,3.0,3.0,3.0,3.0,3.0,3.0
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.395278,6.499802,8.236277,4.0,...,160.0,143.0,0.000003,3.701924,5.0,5.0,5.0,5.0,5.0,5.0
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395278,10.981421,8.534350,91.0,...,3396.0,3026.0,0.000003,0.834126,91.0,91.0,93.0,93.0,93.0,93.0
4,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395278,9.643940,7.709278,29.0,...,948.0,854.0,0.000003,0.835212,21.0,21.0,24.0,24.0,24.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61469,Woolwich Arsenal,Tower Gateway,127,7892,3342,13401.795549,8.973606,8.114328,9.503145,287.0,...,12603.0,11083.0,0.000003,3.596851,293.0,293.0,287.0,287.0,287.0,287.0
61470,Woolwich Arsenal,West Ham,608,7892,5487,8701.454361,8.973606,8.610139,9.071247,124.0,...,5081.0,4495.0,0.000003,0.858426,104.0,104.0,113.0,113.0,112.0,112.0
61471,Woolwich Arsenal,West India Quay,6,7892,400,9536.720451,8.973606,5.991490,9.162906,33.0,...,1382.0,1221.0,0.000003,3.695507,37.0,37.0,35.0,35.0,35.0,35.0
61472,Woolwich Arsenal,West Silvertown,81,7892,893,5355.248554,8.973606,6.794598,8.585834,88.0,...,3317.0,2954.0,0.000003,3.742527,73.0,73.0,79.0,79.0,79.0,79.0


In [709]:
cdatasub_4b_final["flow_diff_10"] = (cdatasub_4b_final["SIM_est_pow_tenth"] - cdatasub_4b_final["SIM_est_pow_old"])
cdatasub_4b_final["flow_diff_100"] = (cdatasub_4b_final["SIM_est_pow_hundred"] - cdatasub_4b_final["SIM_est_pow_old"])


In [710]:
flow_net_4b =cdatasub_4b_final.sort_values(by='flow_diff_10',ascending=False)

In [714]:
flow_net_4b = flow_net_4b[["station_origin","station_destination","flow_diff_10", "flow_diff_100"]]

In [715]:
flow_net_4b.head(10)

,station_origin,station_destination,flow_diff_10,flow_diff_100
32629,London Bridge,Bank and Monument,300.0,333.0
32326,Liverpool Street,Bank and Monument,296.0,328.0
55949,Waterloo,Bank and Monument,181.0,199.0
2712,Bank and Monument,Liverpool Street,169.0,188.0
49948,Stratford,Canary Wharf,153.0,168.0
56075,Waterloo,London Bridge,146.0,161.0
32486,Liverpool Street,Moorgate,142.0,158.0
56193,Waterloo,Westminster,135.0,150.0
27864,Ilford,Stratford,132.0,146.0
56175,Waterloo,Victoria,130.0,143.0
